# Machine Learning pour prédire la gravité des accidents routiers

Dans ce notebook, vous trouvez notre code d'entraînement et validation automatique d'algorithmes de Machine Learning pour prédire la gravité des accidents de voiture à partir du jeu de données de **2019** de *Bases de données annuelles des accidents corporels de la circulation routière - Années de 2005 à 2019*.

**dataset:** https://www.data.gouv.fr/fr/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/#_ <br>
**tâche:** Classification (Classification multi-classe)<br>
**target variable:** *grav*

Nous disposons de 4 datasets différents (*caracteristiques*,*lieux*,*vehicules*,*usagers*) qui contiennent des variables sur les accidents corporels et on cherche à prédire la variable *grav* de la gravité de l'accident:
- 1: Indemne
- 2: Tué
- 3: Hospitalisé
- 4: Blessé léger

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split 
#pour l'entraînement automatique:
from supervised.automl import AutoML
#pour générer un rapport html des résultats:
import IPython
import markdown

## 1. Importation des données

Nous utilisons le jeux de données obtenu à partir de l'union des 4 jeux de données cités plus haut:

In [4]:
data = pd.read_csv('https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2019/20210317-145928/accidents-corp-2019.csv')

## 2. Preprocessing et feature engineering

On modifie les colonnes *lat* et *long*:

In [5]:
data['lat'] = data['lat'].str.replace(',', '.', regex=True)
data['lat'] = data['lat'].astype(float)
data['long'] = data['long'].str.replace(',', '.', regex=True)
data['long'] = data['long'].astype(float)

On crée deux nouvelles variables donnant le moment de l'accident:

In [6]:
def hour(n):
    n=str(n)
    if len(n)<4:
        n='0'+n
    return int(n[:2])

data['heure']=data['hrmn'].apply(hour)
data['mom_acc']=pd.to_datetime(pd.DataFrame({'year': data['an'],'month': data['mois'],'day': data['jour'],'hour':data['heure']}))

Et une variable pour le jour de la semaine:

In [7]:
data['jour_semaine']=data['mom_acc'].dt.dayofweek

## 3. Feature selection et train/test splitting

On sélectionne les variables qui vont faire partie de notre modèle et on partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [8]:
features = ['place','catu','sexe','trajet','secu1','secu2','secu3','locp',
            'catv','an_nais','mois',
            'occutc','obs','obsm','choc','manv',
            'lum','agg','int','atm','col',
            'catr','circ','vosp','prof','plan',
            'surf','infra','situ','lat','long','heure','mom_acc','jour_semaine']

In [9]:
y = data['grav'].values
X = data[features]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4. AutoML

On entraîne, test et valide automatiquement plusieurs algorithmes de ML sur le dataset avec AutoML (plus d'infos ici: https://supervised.mljar.com/) et on génère un html des métriques et des paramètres du modèle.

In [12]:
automl = AutoML(algorithms=['Random Forest', 'Xgboost', 'Decision Tree', 'Baseline', 'Linear'],mode='Explain',random_state=42,explain_level=0)
#fit model
automl.fit(X_train,y_train)
#predictions
predictions = automl.predict(X_test)
#generate html report
automl.report()

### Preprocessing fait par AutoML

Il n'est pas évident de comprendre, à partir du rapport html qui est généré, quelles tâches de preprocessing sont faites dans la **pipeline de AutoML** qui permet d' aboutir aux résultats observés.  <br>
Pour mieux comprendre les résultats, on peut s'intéresser au fichier `framework.json` de chaque modèle. Par exemple, on peut consulter ce fichier pour le modèle `Xgboost` ici:
https://github.com/etalab-ia/DGML/blob/main/docs/automodels/6af37c98-0933-4ae4-8380-5f63212fb52a/3_Default_Xgboost/framework.json <br>
On observe:
1. L'imputation des **valeurs manquantes** est faite en remplaçant par la valeur médiane
2. Une **transformation des dates** est effectuée
3. Un **encoding** des variables catégorielles est fait en transformant le nom de chaque catégorie en un entier

Ces informations sont disponibles pour tous les algorithmes.